# Code to run the different models

## Initial import of packages

In [1]:
from data_handler import get_dataset
from data_utils import *
import matplotlib.pyplot as plt

from models.model_utils import get_model, get_model_args
from models.armodels import get_baselines
from preprocess_data import *
from models.gcnlstm import get_gcnlstm
from data_utils import *
from graph_utils import *

import tensorflow as tf
from keras.optimizers import Adam

## Data setup

In [2]:
dataset = "Palo Alto"
model_name = "convlstm"

prediction_horizon = 7
num_lags = 60

In [3]:
data = get_dataset(dataset)

/Users/frederikhyttel/miniforge3/envs/ml/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3437: DtypeWarning: Columns (29,30,32) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
[-122.163289  -122.1531172 -122.1429454 -122.1327736 -122.1226018
 -122.11243  ]
[37.422054  37.4275282 37.4330024 37.4384766 37.4439508 37.449425 ]


## Get baseline predictions

In [4]:
Var_forecast, AR_forecast = get_baselines(prediction_horizon, get_raster_map(data, normalized=True), data)

## Setup of CONV or CONV+LSTM models

In [5]:
X_train, y_train, X_test, y_test = process_data(data, model_name, prediction_horizon, num_lags)

In [6]:
model_args = get_model_args(model_name, prediction_horizon, X_train) 
model = get_model(model_name, model_args)

In [7]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed (TimeDistri (None, 60, 5, 5, 16)      160       
_________________________________________________________________
time_distributed_1 (TimeDist (None, 60, 400)           0         
_________________________________________________________________
batch_normalization (BatchNo (None, 60, 400)           1600      
_________________________________________________________________
lstm (LSTM)                  (None, 100)               200400    
_________________________________________________________________
batch_normalization_1 (Batch (None, 100)               400       
_________________________________________________________________
dense (Dense)                (None, 175)               17675     
_________________________________________________________________
reshape (Reshape)            (None, 5, 5, 7)           0

In [8]:
model.compile(loss='mae',  optimizer=Adam(lr=model_args['lr']) ,metrics=["mse",'mae'])

hist = model.fit(X_train, y_train,epochs=1,
          batch_size=60, verbose=2, shuffle=True,
          validation_data=(X_test, y_test))

57/57 - 10s - loss: 0.4295 - mse: 0.3513 - mae: 0.4295 - val_loss: 0.0356 - val_mse: 0.0040 - val_mae: 0.0356


## Setup of T-GCN model

In [9]:
G, adj, X_train, y_train, X_test, y_test = setup_GCN(data, prediction_horizon, num_lags)

In [10]:
model = get_gcnlstm(prediction_horizon, num_lags, adj)
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 47, 60)]          0         
_________________________________________________________________
tf.expand_dims (TFOpLambda)  (None, 47, 60, 1)         0         
_________________________________________________________________
reshape_1 (Reshape)          (None, 47, 60)            0         
_________________________________________________________________
fixed_adjacency_graph_convol (None, 47, 64)            6096      
_________________________________________________________________
fixed_adjacency_graph_convol (None, 47, 64)            6352      
_________________________________________________________________
reshape_2 (Reshape)          (None, 47, 64, 1)         0         
_________________________________________________________________
permute (Permute)            (None, 64, 47, 1)         0     

In [11]:
model.compile(loss='mae',  optimizer=Adam(lr=1e-5) ,metrics=["mse",'mae'])

In [12]:
hist = model.fit(
    X_train,
    y_train,
    epochs=1,
    batch_size=16,
    shuffle=True,
    verbose=2,
    validation_data=(X_test, y_test),
)



211/211 - 3s - loss: 0.1818 - mse: 0.0618 - mae: 0.1818 - val_loss: 0.1232 - val_mse: 0.0396 - val_mae: 0.1232
